# 肺結節の良悪性判定

## 前準備
### 主要パッケージのインポート

In [ ]:
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

### データディレクトリの指定

In [ ]:
DATA_ROOT = pathlib.Path('Data/Images/LIDC_PN')
CLASS_LABELS = ('benign', 'malignant')
IMAGE_EXT = '.png'

### 画像ファイルを基にpd.DataFrameを作成する
画像ファイルは`<クラス名>/<ファイル名>`の形式でデータディレクトリ内に用意されている

In [ ]:
def create_dataset_df(data_root, class_labels, image_ext):
    dfs = []
    for cls, class_label in enumerate(class_labels):
        df = pd.DataFrame(
            [(str(p), class_label, cls)
             for p in data_root.glob(class_label + '/*' + image_ext)],
            columns=['filepath', 'class_label', 'class'])
        dfs.append(df)
    df_dataset = pd.concat(dfs, ignore_index=True)
    return df_dataset


df_dataset = create_dataset_df(DATA_ROOT, CLASS_LABELS, IMAGE_EXT)
assert set(CLASS_LABELS) == set(df_dataset['class_label'].unique(
)), 'Discrepancy between CLASS_LABELS and df_dataset'
display(df_dataset)

### クラスごとの画像数を確認する

In [ ]:
df_dataset['class_label'].value_counts()

### 画像を表示してみる
各クラスからランダムに選択した画像を表示する。

In [ ]:
from PIL import Image


def show_images_each_class(df, n_rows=2, n_cols=5):
    for class_label, group in df.groupby('class_label'):
        print(class_label)
        for i, row in enumerate(group.sample(n=n_rows * n_cols).itertuples()):
            plt.subplot(n_rows, n_cols, i + 1)
            image = Image.open(row.filepath)
            row.filepath
            plt.imshow(image, cmap='gray' if image.mode=='L' else None)
            plt.axis('off')
        plt.tight_layout()
        plt.show()


show_images_each_class(df_dataset)

### ホールドアウト検証用にデータセットを分割する
今回はデータセットの$\frac{2}{3}$を学習用、$\frac{1}{3}$を評価用に使用する。
分割にはsklearnの[StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html)を使う。

In [ ]:
from sklearn.model_selection import StratifiedKFold
K_FOLD = 3
kfold = StratifiedKFold(n_splits=K_FOLD, shuffle=True)
train_index, test_index = next(
    kfold.split(df_dataset['filepath'], df_dataset['class']))

df_train = df_dataset.iloc[train_index]
df_test = df_dataset.iloc[test_index]

### データを読み込む
読み込んだ画像は０から255の値をとるため読み込んだあとに255で割ることで0から1の値をとるようにする

In [ ]:
import tensorflow as tf
IMG_SHAPE = (64, 64, 1)


def load_img(filepath):
    return np.atleast_3d(
        tf.keras.preprocessing.image.load_img(
            filepath,
            color_mode='grayscale' if IMG_SHAPE[2] == 1 else 'rgb',
            target_size=IMG_SHAPE))


train_data = np.stack(
    [load_img(filepath) for filepath in df_train['filepath']])
train_labels = df_train['class']
test_data = np.stack([load_img(filepath) for filepath in df_test['filepath']])
test_labels = df_test['class']

train_data = train_data / 255
test_data = test_data / 255

print('training data', train_data.shape, train_labels.shape,
      train_labels.mean())
print('test data', test_data.shape, test_labels.shape, test_labels.mean())

## ネットワーク作成
今回は画像サイズが小さいためモデルを自作する必要があるが、本来は既存のモデルを流用したほうがよい。
<div class="alert alert-block alert-warning">
<b>注意:</b> BatchNormalizationレイヤーのmomentumのデフォルト値は0.99だが、それだとうまくいかなかったので0.90を指定している。
</div>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential()
model.add(layers.Conv2D(4, 3, activation='relu', input_shape=IMG_SHAPE))
model.add(layers.Conv2D(4, 3, activation='relu'))
model.add(layers.BatchNormalization(momentum=0.90))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(8, 3, activation='relu'))
model.add(layers.Conv2D(8, 3, activation='relu'))
model.add(layers.BatchNormalization(momentum=0.90))
model.add(layers.MaxPooling2D(2))
model.add(layers.Conv2D(16, 3, activation='relu'))
model.add(layers.Conv2D(16, 3, activation='relu'))
model.add(layers.BatchNormalization(momentum=0.90))
model.add(layers.MaxPooling2D(2))
model.add(layers.Dropout(.25))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()

### ネットワーク構造の可視化

In [ ]:
import IPython
from tensorflow.python.keras.utils.vis_utils import plot_model
net_arch = tf.keras.utils.model_to_dot(model,
                                       show_shapes=True,
                                       show_layer_names=False,
                                       rankdir='LR',
                                       dpi=200).create(prog='dot',
                                                       format='png')
IPython.display.display_png(IPython.display.Image(net_arch))

## 学習
### 学習経過をモニタするためのクラスを作成
Notebook上で正しく経過を表示するために作成する。チュートリアルではこのクラスを使用するが、通常はverbose=1か2を使えば問題ない。

In [ ]:
import time
import tqdm


class ProgressBarCallback(tf.keras.callbacks.Callback):
    '''Nested progress with single bar
  '''
    def __init__(self, epochs, n_batches, batch_size, leave=True):
        self.n_epochs = epochs
        self.n_batches = n_batches
        self.batch_size = batch_size
        self.bar = tqdm.tqdm(
            total=self.n_batches,
            leave=leave,
            unit='batch',
            desc='1/{n_epochs} epoch'.format(n_epochs=self.n_epochs))

    def __enter__(self):
        return self

    def on_epoch_begin(self, epoch, logs={}):
        self.logs = []
        self.time_epoch_begin = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.bar.reset()
        summary = pd.DataFrame(self.logs).mean().to_dict()
        str_summary = ','.join(
            ['{}={:.03g}'.format(k, v) for k, v in summary.items()])
        duration = time.time() - self.time_epoch_begin
        duration = '{:.02g}m'.format(
            duration / 60) if duration >= 100 else '{:.02g}s'.format(duration)
        self.bar.set_description(
            '{epoch}/{n_epochs} epoch [{duration}/epoch last_epoch=({summary})]'
            .format(duration=duration,
                    summary=str_summary,
                    epoch=epoch + 1,
                    n_epochs=self.n_epochs))

    def on_batch_begin(self, batch, logs={}):
        pass

    def on_batch_end(self, batch, logs={}):
        self.logs.append(logs)
        self.bar.update(self.batch_size)
        self.bar.set_postfix(logs)

    def __exit__(self, exc_type, exc_value, traceback):
        self.bar.close()

### fit
<div class="alert alert-block alert-warning">
<b>注意:</b> 今回、epoch数は決め打ちしてありますが、本来はvalidationデータを用いて学習を終了させる必要があります。
</div>

In [ ]:
BATCH_SIZE = 8
EPOCHS = 16

with ProgressBarCallback(EPOCHS,
                         len(train_data) // BATCH_SIZE, BATCH_SIZE) as pbar:
    result = model.fit(train_data,
                       train_labels.values,
                       batch_size=BATCH_SIZE,
                       epochs=EPOCHS,
                       shuffle=True,
                       verbose=0,
                       callbacks=[pbar])

### 学習履歴の可視化

In [ ]:
pd.DataFrame(result.history).plot(title='Training history', figsize=(5, 3))
plt.show()

## 評価
### 混同行列
学習できているかを確認するため、まずは学習データでの評価を行う。

In [ ]:
from sklearn import metrics


def evaluate(model, data, labels):
    predictions = tf.nn.sigmoid(model.predict(data)).numpy().squeeze()
    y_pred = predictions > .5
    df_result = pd.DataFrame({
        'truth': labels,
        'pred_proba': predictions,
        'pred_class': y_pred
    })
    cm = metrics.confusion_matrix(df_result['truth'], df_result['pred_class'])
    df_cm = pd.DataFrame(cm, index=CLASS_LABELS, columns=CLASS_LABELS)
    df_cm.index.name, df_cm.columns.name = 'Truth', 'Prediction'
    display(df_cm)
    print('Accuracy = {n} / {d} = {a:.03g}%'.format(n=cm.trace(),
                                                    d=cm.sum(),
                                                    a=100 * cm.trace() /
                                                    cm.sum()))
    return df_result


train_result = evaluate(model, train_data, train_labels)

評価データでの評価を行う。

In [ ]:
test_result = evaluate(model, test_data, test_labels.values)

### ROCカーブ

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(test_result['truth'],
                                         test_result['pred_proba'])
auc = metrics.auc(fpr, tpr)
plt.figure(figsize=(3, 3))
plt.plot(fpr, tpr, label='AUC = {auc:.03g}'.format(auc=auc))
plt.plot((0, 1), (0, 1), zorder=0, color='black', alpha=.1,
         linestyle='-')  # diagonal line
plt.xlabel('1 - Specificity')
plt.ylabel('Sensitivity')
plt.legend(loc='lower right')
plt.show()